In [1]:
### create pickle file with SAX dimer features
## PAA time window = 0.05 #0.25s  # 0.5, 0.1s

import pandas as pd
import numpy as np
import os
import pickle
#import re
import regex as re
import matplotlib.pyplot as plt

In [2]:
## create SAX string representation & dimer features

# STEPS
# for each epoch:
# 1. normalize the data over a window (epoch size)
# 2. convert to PAA - take small segments & represent by the average over the segment
# 3. choose alphabet size & the levels/ threshold for discretisation
# 4. convert to string representation
# 5. vector representation - numbers of different dimers

# discretise over 0.25S = 0.25 * 100 = 25 points; reduces to 120 points
#               0.5s => 50 points; reduces to 60 points
#               0.1 s = 0.1 * 100 = 10 points -> 300 data points
#               0.05s = 5 points -> 600 data points


### 1. normalise the data, 1 epoch (30s, i.e. 3000 data points) at a time ###
# if the SD is very small (less than a threshold), do not normalize the data
def z_norm(y):
    sd = y.std()
    if sd < 0.01:
        y = y
    else:
        y =(y - y.mean())/sd
    
    return y



#### 2. convert to PAA - take a window & represent by the average over the window ####
# discretise over 0.25S = 0.25 * 100 = 25 points  -> results in 120 data points
#               0.5s = 50 points   -> results in 60 data points
#               0.1 s = 0.1 * 100 = 10 points -> 300 data points
#               0.05s = 5 points -> 600 data points
#time_window = 0.5
def get_paa(y, time_window):
    # convert time_window from sec to no. of data points
    time_window = time_window * 100  # 100 is sampling freq
    
    paa = []
    for i in range(0, int(len(y)/time_window)): # len(y)/time_window gives the no. of steps
        # take a segment of data
        arr = y[(i * int(time_window)):((i+1) * int(time_window))]
        # convert to mean & append to list
        paa.append(arr.mean())
    return paa



#### 3. choose alphabet size & the levels/ threshold for discretisation ####
### set this up as a dictionary to choose different alphabet sizes  ###
# let alphabet size = 4;
# cuts: [-0.67, 0, 0.67]



#### 4. convert to string  ####
def get_sax_string(paa):
    sax = []
    
    for i in range(0,len(paa)):
        if paa[i] <= -0.67:
            alpha = 'a'
        elif (paa[i] > -0.67 and paa[i] <=0):
            alpha = 'b'
        elif (paa[i] > 0 and paa[i] <=0.67):
            alpha = 'c'
        else:
            alpha = 'd'
        sax.append(alpha)
    
    # convert sax from list to string format
    sax_string = ''.join(sax)
    return sax_string



#### 5. vector representation - numbers of different dimers ####
# different types of dimers
# aa ab ac ad
# ba bb bc bd
# ca cb cc cd
# da db dc dd
### ways to count ###
# sax_string.count(dim_alpha[i]) => 'bbb' : counts bb as 1  => FIX THIS
# len(re.findall(dim_alpha[i], sax_string)) : counts bb as 1

###  set up a list of the dimers possible  ###
dim_alpha = ['aa','ab','ac','ad','ba','bb','bc','bd','ca','cb','cc','cd','da','db','dc','dd']

def get_sax_dimer_array(sax_string):
    dim_arr = []
    
    for i in range(0, len(dim_alpha)):
        dim_arr.append(len(re.findall(dim_alpha[i], sax_string, overlapped=True)))
    #https://stackoverflow.com/questions/2970520/string-count-with-overlapping-occurrences
    
    #print(dim_arr)
    #ncols = 4
    #im = np.reshape(dim_arr,(-1,ncols)) #x.reshape(1754, 1754)
    #plt.gray()
    #plt.imshow(im)
    return dim_arr



def run_all(y, time_window):
    y = z_norm(y)
    #time_window = 0.5
    paa = get_paa(y, time_window)
    sax_string = get_sax_string(paa)
    sax_feature = get_sax_dimer_array(sax_string)
    ## added the following 2 lines to make the data fraction
    total = sum(sax_feature)
    sax_feature = [x/total for x in sax_feature]
    return sax_feature

In [3]:
#time_window = 0.1 #0.5 #0.25

def get_PSD_from_pickle(pid,day,x):
    df = []
    eeg_class = []
    epochnum = []

    
    for k in list(x['wake'].keys()):
        label = 'wake'  #seg = k.split('e')[0]
        epoch = k
        y = x['wake'][k]
        df.append(run_all(y, time_window))  #x['wake'][k])
        eeg_class.append(label)
        epochnum.append(k)

    for k in list(x['sleep_stage_1'].keys()):
        label = 'sleep_stage_1'   #seg = k.split('e')[0]
        epoch = k
        y = x['sleep_stage_1'][k]
        df.append(run_all(y, time_window))  #x['sleep_stage_1'][k])
        eeg_class.append(label)
        epochnum.append(k)
            
    for k in list(x['sleep_stage_2'].keys()):
        label = 'sleep_stage_2'   #seg = k.split('e')[0]
        epoch = k
        y = x['sleep_stage_2'][k]
        df.append(run_all(y, time_window))  #x['sleep_stage_2'][k])
        eeg_class.append(label)
        epochnum.append(k)

    #print(df)    
    df = pd.DataFrame(df)  ## also, add subject ID
    print(df)
    df['pID'] = pid
    df['day'] = day
    eeg_class = pd.DataFrame(eeg_class, columns=['class_label'])
    epochnum = pd.DataFrame(epochnum, columns=['epoch'])
    
    df = pd.concat([df,epochnum,eeg_class],axis=1)
    
    return df

In [4]:
file_location='C:/Users/DAR9KOR/Desktop/data/HEALTHCARE/EEG/data/eeg_epochs_dec12_2018/raw_data' #C:\\Users\\DAR9KOR\\Desktop\\data\\HEALTHCARE\\EEG\\data\\eeg_epochs_dec12_2018'
os.chdir('C:/Users/DAR9KOR/Desktop/data/HEALTHCARE/EEG/data/eeg_epochs_dec12_2018/raw_data') #'/home/intern_eyecare/Desktop/EEG/raw_data') #C:\\Users\\DAR9KOR\\Desktop\\data\\HEALTHCARE\\EEG\\data\\eeg_epochs_dec12_2018')

files=os.listdir(file_location)
file_list = [f for f in files if f.endswith('.pickle')]   # .txt  # .csv

In [5]:
time_window = 0.1 #0.025  #0.05 #0.1 #0.5 #0.25 
dfull = pd.DataFrame()

for file_val in file_list:
    #print(file_val)
    patient = file_val[2:5]
    day = file_val[5:6]
    
    with open(file_val, 'rb') as f:
        print(file_val)
        x = pickle.load(f)      
        dat = get_PSD_from_pickle(patient,day,x)
    f.close()
        
    dfull = pd.concat([dfull, dat],ignore_index=True)

SC4001E0-PSG-EEG.pickle
      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     33  15   5   4  20  52  18   6   3  25  49  14   1   3  19  32
1     27  12   6   5  15  63  24   3   6  29  51   9   2   1  14  32
2     39  13   1   0  12  63  16   7   2  21  50   9   0   1  15  50
3     37  25   9   2  15  22  23  10  15  15  33  20   6   7  18  42
4     30  22  15   0  14  18  29  10  17  23  40  24   6   8  19  24
5     33  11  13   7  17  43  12   8  12  23  50  10   2   4  20  34
6     32  14   3   6  15  66  16   5   7  19  51   9   0   4  16  36
7     39  12   4   3  12  58  16   3   7  15  69   8   0   3  11  39
8     25   9   5  10  18  57  20   2   3  27  57  12   3   4  17  30
9     29   9   6   3  16  61  25   4   2  34  53  10   0   2  15  30
10    44  17   2   1  15  50  18   2   4  17  39  19   1   0  20  50
11    39  12   4   2  15  50  21   4   3  26  66   7   0   3  11  36
12    31  14   5   3  16  59  19   6   5  24  39  14   1   3  19  41
13    32  

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     22   6   3   6  10  90  22   0   3  24  59   8   2   2  11  31
1     34   9   4   5  10  74  19   2   7  22  60   9   1   0  14  29
2     38  13   2   1  15  67  14   3   1  17  73   8   0   2  11  34
3     45   9   0   1   8  68  20   1   0  21  68   8   2   0   8  40
4     36  27   3   1  26  41  21   3   4  20  42  16   0   3  16  40
5     27   9   3   6   8  72  22   4   2  24  76   5   8   2   5  26
6     33  10   1   5  14  61  24   1   2  28  66   6   0   1  11  36
7     32  11   1   5  13  83  16   2   4  20  69   2   0   0   9  32
8     37  13   8   8  21  41  11   5   7  20  46   9   1   3  18  51
9     36  19   6   7  24  48   9   7   7  16  32  13   1   5  21  48
10    40  11   7   8  23  40   9   4   1  22  49  12   2   3  19  49
11    35  10   3   4  17  77  11   2   1  18  44   8   0   1  13  55
12    38  16   3   1  13  65  15   4   3  16  65  11   4   0  12  33
13    35  14   4   1   9  39  22  

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     27  11   2   4  15  73  20   1   2  24  56  11   0   1  14  38
1     43  14   1   0  14  69  18   1   1  18  54  10   0   1  11  44
2     28  22   5   1  20  38  25   6   6  23  42  20   2   7  18  36
3     30  11   1   6  10  72  19   6   4  20  62   6   5   4   9  34
4     53  12   3   4  15  51  17   0   2  18  51  12   1   2  13  45
5     39  14   9   2  19  48  17   3   5  25  37  13   2   0  16  50
6     38  10   7   1  13  69  18   3   3  25  52   8   1   0  11  40
7     42  11   5   1  11  58  16   3   7  16  56  14   0   2  16  41
8     35  13   1   2  14  67  23   1   3  24  69   6   0   1   9  31
9     31   8   4   2  10  60  27   2   2  29  78  11   1   2  12  20
10    31  12   4   1  14  69  25   2   2  26  60  13   1   2  12  25
11    47  11   3   4  15  54  16   1   3  15  53  12   0   6  11  48
12    40  14   1   3  16  47  17   3   2  22  67   8   0   1  14  44
13    39  16   2   4  16  52  19  

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     25  19   2   2  15  68  26   4   6  25  51  11   2   1  13  29
1     38   9   3   0  11  67  23   2   1  25  66  10   0   3  10  31
2     18  12   1   9  17  71  27   6   2  31  57   4   3   6  10  25
3     37  18   8   3  17  46  22   3  10  23  56   9   2   1  12  32
4     31  15   8   8  22  61  16   5   9  22  43   6   1   5  13  34
5     36  12   3   7  17  54  20   3   4  21  50  14   1   7  16  34
6     27  11   5   9  12  51  21   2   9  23  73   9   3   1  15  28
7     26  10  12   2  20  57  15   4   3  26  59  16   1   4  18  26
8     20  12   7   6  16  70  18   5   5  23  53  12   4   3  16  29
9     14  11   7   6  20  84  22   2   3  30  50   8   1   2  13  26
10    19  16   4  11  17  59  23   5  11  25  49   6   3   3  15  33
11    27  16   4   4  19  60  24   5   4  28  49   7   1   4  12  35
12    28  16   7   7  21  67  13   6   4  20  31  12   5   5  15  42
13    31  16   7   5  18  65  20  

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     36  17   8   4  22  47  13   4   5  18  56  13   2   4  15  35
1     45  13   4   3  14  37  16   2   5  19  65  13   2   0  16  45
2     45  17   4   1  14  30  20   3   7  18  67  12   1   2  13  45
3     38  12   8   4  17  46  15   1   4  20  61  13   3   1  15  41
4     48  11   0   0   6  70  16   2   5  10  48  12   0   2  12  57
5     26   9   0   0   9  95  24   0   0  24  56   6   0   0   6  44
6     41  11   7   8  15  42  13   4   6  16  40  16   4   5  19  52
7     35  11  12   1  16  39  16   7   8  22  63  15   0   5  18  31
8     24  18   7   4  17  49  24   5  12  22  50  12   0   6  14  35
9     45  15   6   3  16  35  22   3   8  20  45  17   0   6  17  41
10    38  14  10   3  16  36  17   8  11  21  35  15   0   7  20  48
11    33  17  11   5  24  29  17   3   9  23  41  18   1   4  21  43
12    46  15   2   1  14  42  22   6   5  23  48  13   0   4  16  42
13    33  20   8   4  15  39  24  

...

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     35  13   4   3  14  34  33   3   4  35  63  13   2   2  15  26
1     29  14   5   1  14  67  24   2   4  25  54  13   1   2  13  31
2     37  11   4   2  14  62  23   1   3  24  50   8   0   3   8  49
3     18  12   7   7  16  75  19   4   3  23  61  11   6   4  11  22
4     38  15   5   2  17  66  16   6   4  21  38  10   2   3  14  42
5     21  17   2   1  14  58  35   6   6  34  41  14   0   4  17  29
6     37  20   6   0  18  27  26   7   7  27  47  15   1   4  17  40
7     44  15   5   3  21  48  15   6   2  23  42   9   0   4  15  47
8     40  18   8   2  21  42  19   5   4  21  35  15   3   6  14  46
9     36  19   4   4  18  41  23   5   6  21  39  13   3   5  13  49
10    22  18   6   4  19  45  27   5   9  30  47  15   0   4  21  27
11    23  18   6   9  27  41  25   0   4  30  58   6   3   3   9  37
12    27  16   6   5  20  65  22   6   4  26  35  10   3   6  12  36
13    28  15   7   5  17  51  22  

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     37  10   2   6  14  51  20   1   3  25  74   5   1   1  10  39
1     14  13   3   2  13  88  29   0   5  28  60  11   1   0  12  20
2     28  10   1   2  12  65  20   1   1  20  98   9   0   2  10  20
3     27  13   3   5  16  67  24   0   4  26  49  12   1   2  14  36
4     46  12   3   1  12  47  25   2   4  26  51   9   0   1  11  49
5     17   6   8   4   7  67  27   4   8  29  68  10   3   2  13  26
6     27  13   7   2  19  50  23   6   3  31  67   9   0   5  13  24
7     25  15   8   2  17  46  20   3   7  25  62  15   2   0  19  33
8     25  16   7   5  19  44  20   3   7  25  74   9   1   1  15  28
9     18  15   9   8  23  48  16   8   4  28  54  12   5   4  18  29
10    17  16  10  10  22  54  11   9  11  20  49  10   3   6  21  30
11    11   6   6   8  13  57  37   1   6  39  74   5   1   6   7  22
12    24  12   3  11  16  42  21   5   8  28  67   8   2   3  20  29
13    23   6  13   7  16  53  14  

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     32  13  10   2  15  42  23   5   9  27  52  16   1   3  19  30
1     39  16   4   3  12  62  27   4   9  26  34  11   2   1  15  34
2     19  11   4   7   7  54  37   4   8  32  62  13   7   5  12  17
3     15  19   8   6  23  42  29   2   6  29  74  11   3   6   9  17
4     28  15   6   1  14  37  30   6   6  29  66  15   2   6  14  24
5     24  19   1   2  15  69  23   6   7  22  40  17   0   4  21  29
6     30  16   6   4  17  44  22   7   7  24  55  15   2   6  18  26
7     18  20   8   7  24  43  24   2  10  27  52  13   1   2  19  29
8     22  24   5   4  25  40  26   8   6  28  53  12   1   8  15  22
9     21  10   7   4  14  50  33   8   4  35  60  13   2  10  12  16
10    29  20   7   3  16  46  21   7  12  19  58  15   3   5  18  20
11    24  11   7   4  15  54  27   6   6  29  60  11   1   8  12  24
12    17  16   3   5  15  53  36   6   6  36  48  14   3   5  17  19
13    19  13  10   6  14  45  31  

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     33  12   5   4  14  65  22   2   5  19  58  13   2   6  10  29
1     49  17   2   2  16  34  25   5   5  25  55  10   0   4  13  37
2     38  18   2  10  26  50  22   1   3  25  34  10   1   5  14  40
3     25  16   6   8  21  55  20   2   7  25  53   9   3   2  15  32
4     20   9   6   8  16  70  26   2   6  29  48  10   1   5  14  29
5     26  25   5   5  24  59  15   4   9  15  45  14   2   4  17  30
6     20  15   7   9  19  71  14   4  10  22  40   8   1   1  19  39
7     20  10   5   9  18  64  20   2   7  25  67   7   0   4  14  27
8     19  14   1  11  18  78  17   3   7  22  47   5   1   2  16  38
9     25  15   7   7  18  56  18   7   8  24  54   8   4   4  14  30
10    19  16   9   5  19  53  29   1   9  29  50  14   1   5  14  26
11    25  16   1  10  18  66  31   0   8  29  43   8   0   5  13  26
12    29  21   3   6  23  43  19   9   5  27  43   8   1   3  19  40
13    32  20   1   7  21  47  27  

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0     31  19   5   2  21  72  17   7   3  22  25  13   2   4  16  40
1     40  13   1   1  13  45  19   5   1  24  79   9   1   1  13  34
2     30  24   2   2  27  52  19   5   1  22  50  12   0   4  15  34
3     40  19   4   4  19  40  23   7   7  24  32  19   1   7  22  31
4     27  19   1   3  17  50  26   6   6  26  48  14   1   4  18  33
5     29  11   6   6  16  53  25   7   6  27  48  10   0  10  13  32
6     46  16   5   1  16  34  17   8   7  22  52  12   0   3  18  42
7     33  18   5   3  20  57  21   4   6  23  40  14   0   4  17  34
8     28  14   8   1  16  49  25   6   4  31  52  16   2   3  18  26
9     39  12   2   5  12  63  24   2   6  24  49   9   1   2  13  36
10    18  17   6   4  16  53  38   7  10  37  37  10   1   7  13  25
11    22  10  10   4  17  65  20   5   6  30  44  14   1   2  20  29
12    29  20   0   5  18  35  35   4   6  34  68   8   1   2  14  20
13    25  10   5   8  14  46  33  

In [6]:
## save data
#pickle.dump(dfull, open('Frac_sax_dimer_0.2s_EEG_all_patients.pkl','wb'))

In [6]:
pickle.dump(dfull, open('sax_dimer_0.1s_EEG_all_patients.pkl','wb'))

In [19]:
# old data
#dfull.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,pID,day,epoch,class_label
0,10,4,6,1,8,19,7,4,3,12,20,7,0,2,9,7,400,1,s1e1,wake
1,7,5,1,3,6,22,14,2,3,14,19,4,0,3,6,10,400,1,s1e2,wake
2,11,5,4,2,6,18,10,2,2,12,13,7,3,1,7,16,400,1,s1e3,wake
3,7,8,4,4,4,13,18,4,7,12,12,5,4,6,3,8,400,1,s1e4,wake
4,2,6,5,2,3,12,22,3,9,16,21,5,1,5,4,3,400,1,s1e5,wake


In [20]:
## save data
#pickle.dump(dfull, open('sax_dimer_0.25s_EEG_all_patients.pkl','wb'))

In [6]:
## frac data
#dfull.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,pID,day,epoch,class_label
0,0.084034,0.033613,0.050420,0.008403,0.067227,0.159664,0.058824,0.033613,0.025210,0.100840,0.168067,0.058824,0.000000,0.016807,0.075630,0.058824,400,1,s1e1,wake
1,0.058824,0.042017,0.008403,0.025210,0.050420,0.184874,0.117647,0.016807,0.025210,0.117647,0.159664,0.033613,0.000000,0.025210,0.050420,0.084034,400,1,s1e2,wake
2,0.092437,0.042017,0.033613,0.016807,0.050420,0.151261,0.084034,0.016807,0.016807,0.100840,0.109244,0.058824,0.025210,0.008403,0.058824,0.134454,400,1,s1e3,wake
3,0.058824,0.067227,0.033613,0.033613,0.033613,0.109244,0.151261,0.033613,0.058824,0.100840,0.100840,0.042017,0.033613,0.050420,0.025210,0.067227,400,1,s1e4,wake
4,0.016807,0.050420,0.042017,0.016807,0.025210,0.100840,0.184874,0.025210,0.075630,0.134454,0.176471,0.042017,0.008403,0.042017,0.033613,0.025210,400,1,s1e5,wake


In [7]:
## save data
#pickle.dump(dfull, open('Frac_sax_dimer_0.25s_EEG_all_patients.pkl','wb'))

In [6]:
#dfull.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,pID,day,epoch,class_label
0,0.050847,0.016949,0.067797,0.016949,0.033898,0.152542,0.118644,0.050847,0.033898,0.152542,0.152542,0.033898,0.033898,0.016949,0.033898,0.033898,400,1,s1e1,wake
1,0.033898,0.084746,0.033898,0.000000,0.033898,0.152542,0.118644,0.067797,0.067797,0.101695,0.169492,0.016949,0.016949,0.050847,0.016949,0.033898,400,1,s1e2,wake
2,0.050847,0.033898,0.050847,0.016949,0.033898,0.084746,0.135593,0.033898,0.050847,0.169492,0.118644,0.067797,0.016949,0.000000,0.101695,0.033898,400,1,s1e3,wake
3,0.050847,0.016949,0.033898,0.016949,0.016949,0.169492,0.135593,0.033898,0.033898,0.118644,0.203390,0.033898,0.000000,0.050847,0.033898,0.050847,400,1,s1e4,wake
4,0.000000,0.016949,0.016949,0.000000,0.000000,0.169492,0.237288,0.000000,0.033898,0.186441,0.271186,0.033898,0.000000,0.016949,0.016949,0.000000,400,1,s1e5,wake


In [7]:
#pickle.dump(dfull, open('Frac_sax_dimer_0.05s_EEG_all_patients.pkl','wb'))

In [6]:
## frac data
#dfull.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,pID,day,epoch,class_label
0,0.110368,0.050167,0.016722,0.013378,0.066890,0.173913,0.060201,0.020067,0.010033,0.083612,0.163880,0.046823,0.003344,0.010033,0.063545,0.107023,400,1,s1e1,wake
1,0.090301,0.040134,0.020067,0.016722,0.050167,0.210702,0.080268,0.010033,0.020067,0.096990,0.170569,0.030100,0.006689,0.003344,0.046823,0.107023,400,1,s1e2,wake
2,0.130435,0.043478,0.003344,0.000000,0.040134,0.210702,0.053512,0.023411,0.006689,0.070234,0.167224,0.030100,0.000000,0.003344,0.050167,0.167224,400,1,s1e3,wake
3,0.123746,0.083612,0.030100,0.006689,0.050167,0.073579,0.076923,0.033445,0.050167,0.050167,0.110368,0.066890,0.020067,0.023411,0.060201,0.140468,400,1,s1e4,wake
4,0.100334,0.073579,0.050167,0.000000,0.046823,0.060201,0.096990,0.033445,0.056856,0.076923,0.133779,0.080268,0.020067,0.026756,0.063545,0.080268,400,1,s1e5,wake


In [7]:
## save data
#pickle.dump(dfull, open('Frac_sax_dimer_0.1s_EEG_all_patients.pkl','wb'))

In [8]:
#dd = pickle.load(open('Frac_sax_dimer_0.1s_EEG_all_patients.pkl', 'rb'))
#dd.to_excel("Frac_sax_dimer_0.1s_EEG_all_patients.xlsx")

In [6]:
## save data
pickle.dump(dfull, open('Frac_sax_dimer_0.15s_EEG_all_patients.pkl','wb'))